In [ ]:
!pip install bitsandbytes accelerate --upgrade transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login() 

In [ ]:
import transformers
import torch
from transformers import AutoModelForCausalLM

In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
    trust_remote_code=True
)

messages = [
    {"role": "system", "content": "You are an expert Ai Scientist"},
]


In [ ]:
while True:
    user_input = input("You: ")
    messages.append({"role": "user", "content": user_input})

    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    outputs = pipeline(
        prompt,
        max_new_tokens=200,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    response = outputs[0]["generated_text"][len(prompt):]
    print(f"AI: {response}")

    messages.append({"role": "assistant", "content": response})
